# Convolução

In [16]:
import torch
from torch import nn
import torch.nn.functional as F

In [17]:
x = torch.tensor([5, 4, 8, 7, 9, 3, 6], dtype=torch.float32)
weight = torch.tensor([1, 2, 3], dtype=torch.float32)

# kernel size (tamanho do filtro)
ks = len(weight)

# Adiciona a dimensão do batch e do canal. Final: 1 x 1 x len(x)
x = x.reshape(1, 1, len(x))
print(x.shape)

torch.Size([1, 1, 7])


Convolução 1D


In [18]:
# Reshape para adicionar 2 dimensões
weight = weight.reshape(1, 1, ks)
print(weight.shape)


# Convolução

# Caso não seja adicionado padding nos dados, o sinal diminuirá de dimensão 
# Realiza a multiplicação dos elementos e soma
y = F.conv1d(x, weight, padding=ks//2)
y.shape

torch.Size([1, 1, 3])


torch.Size([1, 1, 7])

In [19]:
y

tensor([[[22., 37., 41., 49., 34., 33., 15.]]])

Adicionando bias

In [20]:
bias = torch.tensor([5.])
y = F.conv1d(x, weight, padding=ks//2, bias=bias)
y

tensor([[[27., 42., 46., 54., 39., 38., 20.]]])

Replicando a convolução 1d com o método com camadas de convoluções

In [24]:
conv = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=ks, padding=ks//2, bias=False)
# filtro está armazenado no atributo conv.weight
with torch.no_grad():
    conv.weight[:] = weight
conv.weight

Parameter containing:
tensor([[[1., 2., 3.]]], requires_grad=True)

In [25]:
y = conv(x)
y


tensor([[[22., 37., 41., 49., 34., 33., 15.]]], grad_fn=<ConvolutionBackward0>)

A convolução pode ser implementada como um produto matricial!


Ou seja, a convolução 1D pode ser realizada com uma camada linear:

Mesmo resultado.

In [26]:
matrix = torch.tensor([
    [2, 3, 0, 0, 0, 0, 0],
    [1, 2, 3, 0, 0, 0, 0],
    [0, 1, 2, 3, 0, 0, 0],
    [0, 0, 1, 2, 3, 0, 0],
    [0, 0, 0, 1, 2, 3, 0],
    [0, 0, 0, 0, 1, 2, 3],
    [0, 0, 0, 0, 0, 1, 2],
], dtype=torch.float32)

x = torch.tensor([5, 4, 8, 7, 9, 3, 6], dtype=torch.float32)
F.linear(x, matrix)

tensor([22., 37., 41., 49., 34., 33., 15.])

# Convolução com mais de um canal

In [36]:
# 1 canal, 4 camadas e 7 imagens?
x = torch.rand(size=(1,4,7))
print(x)
conv = nn.Conv1d(in_channels=4, out_channels=5, kernel_size=ks, padding=ks//2, bias=False)


y = conv(x)
print("\nCanal | Quantidade de canais de entrada | tamanho dos dados")
y.shape

tensor([[[0.9806, 0.3803, 0.4162, 0.8720, 0.2542, 0.6749, 0.7670],
         [0.6918, 0.9558, 0.9288, 0.6618, 0.2476, 0.9095, 0.1046],
         [0.2241, 0.3956, 0.2717, 0.5412, 0.5017, 0.0506, 0.4630],
         [0.1812, 0.6278, 0.7643, 0.6495, 0.6421, 0.8452, 0.3997]]])

Canal | Quantidade de canais de entrada | tamanho dos dados


torch.Size([1, 5, 7])

## Filtros 

* O número de canais de saída $C_out$ define o numero de filtros da camada

* Cada filtro possui tamanho espacial ks

* Cada filtro possui profundidade (numero de canais) igual ao tensor de entrada



In [31]:

# [canais de saida, canal de entrada, profundidade da imagem]
conv.weight.shape

torch.Size([5, 4, 3])

In [29]:
#4 canais e tamanho espacial 3
conv.weight[0].shape

torch.Size([4, 3])

## Stride

O stride define o passo do filtro.
Por exemplo, stride=2 pula de 2 em 2 elementos a cada etapa


In [37]:
x = torch.tensor([5, 4, 8, 7, 9, 3, 6], dtype=torch.float32)
weight = torch.tensor([1, 2, 3], dtype=torch.float32)

# kernel size (tamanho do filtro)
ks = len(weight)

# Adiciona a dimensão do batch e do canal. Final: 1 x 1 x len(x)
x = x.reshape(1, 1, len(x))
weight = weight.reshape(1, 1, ks)

y = F.conv1d(x, weight, stride=2)
y

tensor([[[37., 49., 33.]]])

Stride 2 pode ser utilizado para reduzir o sinal progressivamente até um tamanho desejado para utilizar na classificação da imagem

In [38]:
y = F.conv1d(x, weight, stride=1)
y[0,0::2]

tensor([[37., 41., 49., 34., 33.]])

## Dilatação

In [43]:
y = F.conv1d(x, weight, dilation=2)
y

tensor([[[48., 27., 44.]]])

Aumenta o tamanho do filtro (sem aumentar a quantidade de parâmetros) intercalando os pesos do filtro com 0. 

Utilizado para a tarefa **segmentação de imagens** principalmente, quando o contexto do pixel pode ser importante.

In [45]:

weight_d = torch.tensor([1, 0, 2, 0, 3], dtype=torch.float32).reshape(1, 1, -1)
F.conv1d(x, weight_d)

tensor([[[48., 27., 44.]]])